# A2A 클라이언트 튜토리얼

먼저 에이전트 서버 실행:
```bash
uv run python tutorial/text_to_sql/a2a_agent.py  # 포트 8001
uv run python tutorial/search_agent/a2a_agent.py  # 포트 8002
```

In [1]:
import uuid
import httpx
from a2a.client import A2AClient
from a2a.types import SendMessageRequest, MessageSendParams, Message, TextPart

In [2]:
def create_request(text: str) -> SendMessageRequest:
    return SendMessageRequest(
        id=str(uuid.uuid4()),
        params=MessageSendParams(
            message=Message(
                role="user",
                messageId=str(uuid.uuid4()),
                parts=[TextPart(kind="text", text=text)],
            )
        )
    )

In [3]:
def get_response_text(result) -> str:
    response = result.root if hasattr(result, "root") else result
    if hasattr(response, "error") and response.error:
        return f"Error: {response.error}"
    
    task = getattr(response, "result", None)
    if not task or not hasattr(task, "history"):
        return str(response)
    
    for msg in task.history:
        role = msg.role.value if hasattr(msg.role, "value") else msg.role
        if role == "agent":
            for part in msg.parts:
                if hasattr(part, "root") and hasattr(part.root, "text"):
                    return part.root.text
                if hasattr(part, "text"):
                    return part.text
    return ""

## 에이전트 연결

In [4]:
http = httpx.AsyncClient(timeout=120.0)
search_client = A2AClient(httpx_client=http, url="http://localhost:8002")
search_card = await search_client.get_card()
print(f"Search Agent: {search_card.name}")

Search Agent: Search Agent


/var/folders/xq/1pbhtmz5717g9kb31s4dbsfm0000gn/T/ipykernel_93663/2704181881.py:2: DeprecationWarning: A2AClient is deprecated and will be removed in a future version. Use ClientFactory to create a client with a JSON-RPC transport.
  search_client = A2AClient(httpx_client=http, url="http://localhost:8002")


In [5]:
sql_client = A2AClient(httpx_client=http, url="http://localhost:8001")
sql_card = await sql_client.get_card()
print(f"SQL Agent: {sql_card.name}")

SQL Agent: Text-to-SQL Agent


/var/folders/xq/1pbhtmz5717g9kb31s4dbsfm0000gn/T/ipykernel_93663/778463828.py:1: DeprecationWarning: A2AClient is deprecated and will be removed in a future version. Use ClientFactory to create a client with a JSON-RPC transport.
  sql_client = A2AClient(httpx_client=http, url="http://localhost:8001")


## Text-to-SQL Agent

In [6]:
result = await sql_client.send_message(create_request("어떤 테이블들이 있어?"))
print(get_response_text(result))

데이터베이스에는 다음과 같은 테이블들이 있습니다:

- Album
- Artist
- Customer
- Employee
- Genre
- Invoice
- InvoiceLine
- MediaType
- Playlist
- PlaylistTrack
- Track

각 테이블은 디지털 미디어 스토어의 다양한 정보를 담고 있습니다. 궁금한 테이블이 있으면 말씀해 주세요!


In [7]:
result = await sql_client.send_message(create_request("가장 인기있는 아티스트 찾고 웹에서 정보 검색해줘"))
print(get_response_text(result))

가장 인기 있는 아티스트는 Iron Maiden입니다. (트랙 수 기준, 213곡)

Iron Maiden은 1975년 영국 런던에서 스티브 해리스가 결성한 세계적인 헤비 메탈 밴드입니다. 대표 멤버로는 스티브 해리스(베이스), 브루스 디킨슨(보컬), 데이브 머레이, 에이드리언 스미스, 야닉 거스(이상 기타), 니코 맥브레인(드럼)이 있습니다.

이들은 ‘The Number of the Beast’ 등 17장의 정규 앨범과 수많은 히트곡을 발표했으며, 뉴 웨이브 오브 브리티시 헤비 메탈(NWOBHM) 운동을 이끈 대표 밴드로 평가받습니다. 40년이 넘는 활동 기간 동안 강렬한 연주와 독창적인 무대 연출로 전 세계 록 팬들에게 큰 사랑을 받고 있습니다.

자세한 정보는 아래 링크에서 확인할 수 있습니다:
- [Iron Maiden - Wikipedia](https://ko.wikipedia.org/wiki/Iron_Maiden)
- [Iron Maiden - 벅스](https://music.bugs.co.kr/artist/8001)
- [Iron Maiden - maniadb](http://www.maniadb.com/artist/106203)
- [아이언 메이든 - IZM](http://izm.co.kr/contentRead.asp?idx=1669&bigcateidx=11&subcateidx=13&view_tp=1)


## Search Agent

In [ ]:
result = await search_client.send_message(create_request("2026 AI 트렌드"))
print(get_response_text(result))